In [3]:
import numpy as np
import cv2
import os
from random import shuffle
from tqdm import tqdm 

In [4]:
DATA_DIR = "K:\Research\MyMachli\Dataset\machli_image"
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'fishCNN-{}-{}.model'.format(LR, '2conv-basic')

def createData():
    data = []
    for dir in tqdm(os.listdir(DATA_DIR)):
        img_path = os.path.join(DATA_DIR, dir)
        label = dir.split('_')[-1]
        oneHotLabel = np.zeros(23)
        oneHotLabel[int(label)-1] = 1
        for img in tqdm(os.listdir(img_path)):
            path = os.path.join(img_path, img)
            img = cv2.imread(path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            data.append([np.array(img), np.array(oneHotLabel)])
        shuffle(data)
        np.save('data.npy',data)
        return data

In [ ]:
data = createData()
# data = np.load("data.npy")
print(len(data))

  9%|██████▉                                                                     | 1096/12112 [00:29<01:49, 100.53it/s]


 36%|███████████████████████████▍                                                | 4375/12112 [00:40<00:33, 233.18it/s]


 44%|█████████████████████████████████                                           | 5269/12112 [00:58<01:03, 108.21it/s]


 46%|███████████████████████████████████▊                                         | 5631/12112 [01:14<08:26, 12.79it/s]


 48%|████████████████████████████████████▌                                        | 5757/12112 [01:28<28:37,  3.70it/s]


 50%|██████████████████████████████████████▎                                      | 6029/12112 [01:46<04:20, 23.32it/s]


 66%|██████████████████████████████████████████████████▍                         | 8038/12112 [01:55<00:13, 297.96it/s]


 79%|█████████████████████████████████████████████████████████████                | 9597/12112 [02:22<11:59,  3.50it/s]


 94%|██████████████████████████████████████████████████████████████████████▍    | 11377/12112 [02:37<00:04, 177.90it/s]


100%|████████████████████████████████████████████████████████████████████████████| 12112/12112 [02:42<00:00, 74.35it/s]

In [4]:
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 23, activation = 'softmax')
convnet = regression(convnet, optimizer = 'adam', learning_rate=LR, loss = 'categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir = 'log')

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


In [5]:
if(os.path.exists('{}.meta'.format(MODEL_NAME))):
    model.load(MODEL_NAME)
    print('model successfully loaded')

In [6]:
train = data[:-2000]
test = data[23000:]

In [ ]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
test_y = [i[1] for i in test]

test_x.size

90000000

In [ ]:
model.fit({'input':X}, {'targets': Y}, n_epoch=5, validation_set = ({'input':test_x}, {'targets':                                
                                                            test_y}), snapshot_step=500, show_metric = True, run_id = MODEL_NAME)

---------------------------------
Run id: fishCNN-0.001-2conv-basic.model
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 9112
Validation samples: 3000
--
